## Load a Rubra function calling model

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "sanjay920/Llama-3-8b-function-calling-alpha-v1"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)




/home/acorn/anaconda3/envs/myvllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.13it/s]


## Define functions

In [2]:
functions = [
        {
            "type": "function",
            "function": {
                "name": "list_files",
                "description": "List all files in a directory",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "directory": {
                            "type": "string",
                            "description": "the directory to list files from"
                        }
                    },
                    "required": [
                        "directory"
                    ]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "description": "Create a 3D model of an object with specified dimensions",
                "name": "create_3d_model",
                "parameters": {
                    "properties": {
                    "object_name": {
                        "description": "Name of the object to be modeled",
                        "type": "string"
                    },
                    "dimensions": {
                        "description": "Dimensions of the 3D object (length, width, height)",
                        "type": "object",
                        "properties": {
                        "length": {
                            "type": "number"
                        },
                        "width": {
                            "type": "number"
                        },
                        "height": {
                            "type": "number"
                        }
                        },
                        "required": [
                        "length",
                        "width",
                        "height"
                        ]
                    }
                    },
                    "required": [
                    "object_name",
                    "dimensions"
                    ],
                    "type": "object"
                }
            }
        },
        {
            "type": "function",
            "function": {
                "description": "Get the latest insurance premium from a list of premiums.",
                "name": "latest_insurance_premium",
                "parameters": {
                    "properties": {
                        "premiums": {
                            "description": "List of insurance premiums",
                            "type": "array",
                            "items": {
                            "type": "number"
                            }
                        }
                    },
                    "required": [
                        "premiums"
                    ],
                    "type": "object"
                }
            }
        },
        {
            "type": "function",
            "function": {
            "description": "Calculate insurance premium based on age and coverage",
            "name": "calculate_insurance_premium",
            "parameters": {
                "properties": {
                "age": {
                    "description": "Age of the person applying for insurance",
                    "type": "integer"
                },
                "coverage_type": {
                    "description": "Type of insurance coverage",
                    "type": "string",
                    "enum": [
                    "basic",
                    "standard",
                    "premium"
                    ]
                }
                },
                "required": [
                "age",
                "coverage_type"
                ],
                "type": "object"
            }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_file_size",
                "description": "Get the size of a file with given a filename",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "filename": {
                            "type": "string",
                            "description": "the name of the file to get its size"
                        }
                    },
                    "required": [
                        "filename"
                    ]
                }
            }
        },
        {
            'type': 'function',
            'function': {
                'name': 'addition',
                'description': "Adds two numbers together",
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'a': {
                            'description': 'First number to add',
                            'type': 'string'
                        },
                        'b': {
                            'description': 'Second number to add',
                            'type': 'string'
                        }
                    },
                    'required': []
                }
            }
        },
        {
            'type': 'function',
            'function': {
                'name': 'subtraction',
                'description': "Subtracts two numbers",
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'a': {
                            'description': 'First number to be subtracted from',
                            'type': 'string'
                        },
                        'b': {
                            'description': 'Number to subtract',
                            'type': 'string'
                        }
                    },
                    'required': []
                }
            }
        },
        {
            'type': 'function',
            'function': {
                'name': 'multiplication',
                'description': "Multiply two numbers together",
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'a': {
                            'description': 'First number to multiply',
                            'type': 'string'
                        },
                        'b': {
                            'description': 'Second number to multiply',
                            'type': 'string'
                        }
                    },
                    'required': []
                }
            }
        },
        {
            'type': 'function',
            'function': {
                'name': 'division',
                'description': "Divide two numbers",
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'a': {
                            'description': 'First number to use as the dividend',
                            'type': 'string'
                        },
                        'b': {
                            'description': 'Second number to use as the divisor',
                            'type': 'string'
                        }
                    },
                    'required': []
                }
            }
        },
      {
          "type": "function",
        "function": {
          "name": "getCurrentWeather",
          "description": "Get the weather in location",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
              "unit": {"type": "string", "enum": ["c", "f"]}
            },
            "required": ["location"]
          }
        }
      },
      {    "type": "function",
        "function":
        {
            "name": "orderUmbrella",
            "description": "Do this to help user to order an umbrella online", 
            "parameters": {
                "type": "object",
                "properties": {
                    "number_to_buy": {
                        "type": "integer",
                        "description": "the amount of umbrellas to buy"
                    }
                },
                "required": [
                    "number_to_buy"
                ]
            }
        }},
      {"type": "function","function":{"name":"calculate_distance","description":"Calculate the distance between two locations","parameters":{"type":"object","properties":{"origin":{"type":"string","description":"The starting location"},"destination":{"type":"string","description":"The destination location"},"mode":{"type":"string","description":"The mode of transportation"}},"required":["origin","destination","mode"]}}},{"type": "function","function":{"name":"generate_password","description":"Generate a random password","parameters":{"type":"object","properties":{"length":{"type":"integer","description":"The length of the password"}},"required":["length"]}}}
       ]

## Start Conversation:

In [4]:
from llama_tools import preprocess_input, postprocess_output

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the result of  four plus six? Take the result and add 2? Then multiply by 5 and then divide by two"},
]

def run_model(messages, functions):
    ## Format messages in Rubra's format
    formatted_msgs = preprocess_input(msgs=messages, tools=functions)

    input_ids = tokenizer.apply_chat_template(
        formatted_msgs,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=1000,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    raw_output = tokenizer.decode(response, skip_special_tokens=True)
    return raw_output

raw_output = run_model(messages, functions)
# Check if there's a function call
function_call = postprocess_output(raw_output)
if function_call:
    print(function_call)
else:
    print(raw_output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


[{'id': 'fc65a533', 'function': {'name': 'addition', 'arguments': '{"a": "4", "b": "6"}'}, 'type': 'function'}]


## Continue Conversation with tool call observation

In [5]:
if function_call:
    # append the assistant tool call msg
    messages.append({"role": "assistant", "tool_calls": function_call})
    # append the result of the tool call in openai format, in this case, the value of adding 6 to 4 is 10.
    messages.append({'role': 'tool', 'tool_call_id': function_call[0]["id"], 'name': function_call[0]["function"]["name"], 'content': '10'})
    raw_output = run_model(messages, functions)
    # Check if there's a function call
    function_call = postprocess_output(raw_output)
    if function_call:
        print(function_call)
    else:
        print(raw_output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


[{'id': '2ffc3de4', 'function': {'name': 'addition', 'arguments': '{"a": "10", "b": "2"}'}, 'type': 'function'}]
